<a href="https://colab.research.google.com/github/alirezash97/Pattern-Recognition-Course/blob/main/HCC%20Survival/HCC_Survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00423/hcc-survival.zip'

In [2]:
!unzip '/content/hcc-survival.zip'

Archive:  /content/hcc-survival.zip
   creating: hcc-survival/
  inflating: hcc-survival/hcc-data.txt  
   creating: __MACOSX/
   creating: __MACOSX/hcc-survival/
  inflating: __MACOSX/hcc-survival/._hcc-data.txt  
  inflating: hcc-survival/hcc-description.txt  
  inflating: __MACOSX/hcc-survival/._hcc-description.txt  


In [1]:
import pandas as pd 

dataset = pd.read_csv (r'/content/hcc-survival/hcc-data.txt', header=None)
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,1,0,1,0,0,0,0,1,0,1,1,?,1,0,0,0,0,1,0,0,0,0,1,67,137,15,0,1,1,1.53,95,13.7,106.6,4.9,99,3.4,2.1,34,41,183,150,7.1,0.7,1,3.5,0.5,?,?,?,1
1,0,?,0,0,0,0,1,1,?,?,1,0,0,1,0,0,0,1,0,0,0,0,1,62,0,?,0,1,1,?,?,?,?,?,?,?,?,?,?,?,?,?,?,1,1.8,?,?,?,?,1
2,1,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1,0,1,1,78,50,50,2,1,2,0.96,5.8,8.9,79.8,8.4,472,3.3,0.4,58,68,202,109,7,2.1,5,13,0.1,28,6,16,1
3,1,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,77,40,30,0,1,1,0.95,2440,13.4,97.1,9,279,3.7,0.4,16,64,94,174,8.1,1.11,2,15.7,0.2,?,?,?,0
4,1,1,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,76,100,30,0,1,1,0.94,49,14.3,95.1,6.4,199,4.1,0.7,147,306,173,109,6.9,1.8,1,9,?,59,15,22,1


In [2]:
import numpy as np 


dataset = dataset.replace(['?'], np.nan)
dataset.isna().sum()

0      0
1     18
2      0
3     17
4     39
5     24
6      9
7      0
8     39
9     41
10     3
11    10
12    23
13     3
14     2
15    14
16    22
17    52
18    15
19    11
20     3
21     4
22     2
23     0
24    48
25    53
26     0
27     1
28     2
29     4
30     8
31     3
32     3
33     3
34     3
35     6
36     5
37     4
38     3
39     3
40     3
41    11
42     7
43     2
44    20
45    44
46    79
47    80
48    80
49     0
dtype: int64

In [3]:
missing_values_indexes = {}
for column in dataset:
  if dataset[column].isnull().values.any():
    missing_values_indexes[str(column)] = dataset[column].isnull().to_numpy().nonzero()

In [4]:
def missing_handle_classmean(dataframe, column, indexes):
  

  for index in indexes[0]:
    
    missing_value_class_values = []
  
    index_label = dataframe[49][index]
    for row_index, row in dataset.iterrows():
      
      row_label = dataframe[49][row_index]
      if str(index_label) == str(row_label):
        if (dataframe[int(column)][row_index] is None) or (str(dataframe[int(column)][row_index]) == 'nan'):
          pass
        else:
          missing_value_class_values.append( int(float(dataframe[int(column)][row_index])) )

    if missing_value_class_values:
      dataframe[int(column)][index] = np.mean(missing_value_class_values)
    else:
      dataframe[int(column)][index] = 0

  
  return dataframe

In [5]:
def missing_handle_mean(dataframe, column, indexes):
  
  for index in indexes[0]:

    
    column_without_nan = dataframe[int(column)].dropna()
    column_without_nan = np.array(np.array(column_without_nan.values, dtype=float), dtype=int)
    dataframe[int(column)][index] = column_without_nan.mean()
  
  return dataframe

In [6]:

def missing_handler(dataframe, missing_values_indexes, method='Class_mean'):
  

  if method=='Class_mean':
    for key in missing_values_indexes:
      dataframe = missing_handle_classmean(dataframe, key, missing_values_indexes[key])

  elif method=='feature_mean':
    for key in missing_values_indexes:
      dataframe = missing_handle_mean(dataframe, key, missing_values_indexes[key])
  else:
    pass

  
  return dataframe

In [7]:
dataset = missing_handler(dataset, missing_values_indexes, method='Class_mean')
dataset.isna().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
dtype: int64

In [8]:
def split_dataset(df, train_percentage=0.8, test_percentage=0.2):



  train = df.sample(frac=(1-test_percentage),random_state=200) #random state is a seed value
  test = df.drop(train.index)

  return train, test


In [9]:
trainset, testset = split_dataset(dataset, 0.8, 0.2)
print(len(trainset))
print(len(testset))

132
33


In [10]:
x_train = trainset.drop([49], axis = 1).values
y_train = trainset.loc[:, [49]].values.astype('int')

x_test = testset.drop([49], axis = 1).values
y_test = testset.loc[:, [49]].values.astype('int')

In [11]:
print('x_train shapes: ', x_train.shape)
print('y_train shapes: ', y_train.shape)
print('x_test shapes: ', x_test.shape)
print('y_test shapes: ', y_test.shape)

x_train shapes:  (132, 49)
y_train shapes:  (132, 1)
x_test shapes:  (33, 49)
y_test shapes:  (33, 1)


In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
print(clf.predict(x_test))
print('-----------------')
print(y_test)

[1 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1]
-----------------
[[1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [14]:
clf.score(x_test, y_test)

0.8787878787878788

In [ ]:
# accuracy score on test data is 0.87 using << the mean of the class >> while 
# it is 0.74 using << the mean of column >> 